- All non-negative coefficients
- Bounds for Intercept depend on no_intercept flag also
- Specific order for some coefficients
- Min percentage by which the coefficients (of ordered features) differ

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy.optimize import lsq_linear

# User inputs

In [2]:
df = pd.read_csv("data/wages.csv")

In [3]:
no_intercept = True

In [4]:
target = 'TotalWageCost_Values'

In [5]:
# Features in the expected ascending order of coefficients
features = ['HeadCount_DS_1', 'HeadCount_DS_2', 'HeadCount_DS_3', 'HeadCount_DS_4', 'HeadCount_DS_5']
features

['HeadCount_DS_1',
 'HeadCount_DS_2',
 'HeadCount_DS_3',
 'HeadCount_DS_4',
 'HeadCount_DS_5']

In [6]:
# Min percentage gaps between successive (ordered) features
min_gap_pct = [None, 0.13, 0.51, 0.09, 0.03]

# Constraints

In [7]:
# Initialize coefficients
len_coeffs = len(features) + 1
coeffs = list(np.zeros(len_coeffs))
print("Initialized coefficients:", coeffs)

Initialized coefficients: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
# Put constraints 
min_con_orig = [0, 56, 64, 108, 97, 111]
max_con_orig = [np.inf, 95, 106, 171, 160, 176]

if no_intercept:
    min_con_orig[0] = 0
    max_con_orig[0] = 0.0001

In [9]:
# Sanitize constraints
for i in range(2, len(features) + 1):
    min_con_orig[i] = max((1 + min_gap_pct[i - 1]) * min_con_orig[i - 1], min_con_orig[i])

i = len(features) - 1
while i:
    max_con_orig[i] = min(max_con_orig[i + 1] / (1 + min_gap_pct[i]), max_con_orig[i])
    i -= 1
    
print("min_con_orig", min_con_orig)
print("max_con_orig", max_con_orig)

for i in range(len_coeffs):
    if max_con_orig[i] < min_con_orig[i]:
        assert(0)

min_con_orig [0, 56, 64, 108, 117.72000000000001, 121.25160000000001]
max_con_orig [0.0001, 86.02765681632074, 97.21125220244244, 146.78899082568807, 160, 176]


In [10]:
focal_coeff_idx = 5

In [11]:
min_con = min_con_orig.copy()
max_con = max_con_orig.copy()

for i in range(focal_coeff_idx + 1, len(features) + 1):
    min_con[i] = max(0, min_con_orig[i] - max_con_orig[i - 1] * (1 + min_gap_pct[i - 1]))
    max_con[i] = max(min_con[i] + 0.001, max_con_orig[i] - min_con_orig[i - 1] * (1 + min_gap_pct[i - 1]))
    
i = focal_coeff_idx - 1
while i:
    min_con[i] = max(0, min_con_orig[i + 1] / (1 + min_gap_pct[i]) - max_con_orig[i])
    max_con[i] = max(min_con[i] + 0.001, max_con_orig[i + 1] / (1 + min_gap_pct[i]) - min_con_orig[i])
    i -= 1

print("min_con", min_con)
print("max_con", max_con)

min_con [0, 0, 0, 0, 0, 121.25160000000001]
max_con [0.0001, 30.027656816320743, 33.21125220244244, 38.788990825688074, 53.15378640776699, 176]


# Model

y = X0 + 
    X3*DS_3 + 
    (X3*1.09+X4)*DS_4 + ((X3*1.09+X4)*1.03+X5)*DS_5 + 
    (X3/1.51-X2)*DS_2 + ((X3/1.51-X2)/1.13-X1)*DS_1

y = X0 + 
    X5*DS_5 + X4*(DS_5*1.03+DS_4) + 
    X1*(-DS_1) + X2*(-DS_1/1.13-DS_2) + 
    X3*(DS_3+DS_4*1.09+DS_5*(1.09*1.03)+DS_2/1.51+DS_1/(1.51/1.13))

In [12]:
# Feature engineer   
X = df[features].copy()

i = len(features)
while i > focal_coeff_idx:
    X[f'F{i}'] = X[features[i - 1]]
    if i < len(features):
        X[f'F{i}'] += X[f'F{i + 1}'] * (1 + min_gap_pct[i])
    i -= 1

for i in range(1, focal_coeff_idx):
    X[f'F{i}'] = -X[features[i - 1]]
    if i > 1:
        X[f'F{i}'] += X[f'F{i - 1}'] / (1 + min_gap_pct[i - 1])

X[f'F{focal_coeff_idx}'] = X[features[focal_coeff_idx - 1]]
if focal_coeff_idx < len(features):
    X[f'F{focal_coeff_idx}'] += X[f'F{focal_coeff_idx + 1}'] * (1 + min_gap_pct[focal_coeff_idx])
if focal_coeff_idx > 1:
    X[f'F{focal_coeff_idx}'] -= X[f'F{focal_coeff_idx - 1}'] / (1 + min_gap_pct[focal_coeff_idx - 1])
                           
X = X.drop(features, axis=1)

In [13]:
# Convert independent variables to a matrix
X = X.values

# Add an array of ones to act as intercept coefficient
ones = np.ones(X.shape[0])
# Combine array of ones and indepedent variables
X = np.concatenate((ones[:, np.newaxis], X), axis=1)
X

array([[  1.        , -10.        , -16.84955752, -17.15864737,
        -19.74187832,  21.16687215],
       [  1.        ,  -9.        , -14.96460177, -14.9103323 ,
        -16.67920394,  17.19340189],
       [  1.        , -11.        , -18.73451327, -19.40696243,
        -22.80455269,  25.14034242],
       [  1.        , -10.        , -16.84955752, -17.15864737,
        -19.74187832,  21.16687215],
       [  1.        , -12.        , -20.61946903, -21.6552775 ,
        -25.86722707,  29.11381268],
       [  1.        , -11.        , -18.73451327, -19.40696243,
        -22.80455269,  25.14034242],
       [  1.        , -13.        , -22.50442478, -23.90359257,
        -28.92990144,  33.08728295],
       [  1.        , -12.        , -20.61946903, -21.6552775 ,
        -25.86722707,  29.11381268],
       [  1.        , -14.        , -24.38938053, -26.15190764,
        -31.99257581,  37.06075322],
       [  1.        , -13.        , -22.50442478, -23.90359257,
        -28.92990144,  33.0

In [14]:
# Convert target variable to a matrix
y = df[target].values
y

array([3107, 2538, 3647, 3107, 4243, 3647, 4828, 4243, 5391, 4828, 5965,
       5391, 6575, 5965, 7108, 6575, 7724, 7108])

In [15]:
# Run optimization
results = lsq_linear(X, y, bounds=(min_con, max_con), lsmr_tol='auto')
print("Results:\n", results)

Results:
  active_mask: array([ 1, -1, -1, -1, -1,  0])
        cost: 2761.2421558211554
         fun: array([-24.10749773, -33.83170438,  14.61670893, -24.10749773,
        -2.65908442,  14.61670893,  -8.93487776,  -2.65908442,
         6.7893289 ,  -8.93487776,  11.51353555,   6.7893289 ,
       -19.76225779,  11.51353555,  25.96194886, -19.76225779,
       -11.31384448,  25.96194886])
     message: 'The relative change of the cost function is less than `tol`.'
         nit: 14
  optimality: 1.0372898452254649e-08
      status: 2
     success: True
           x: array([1.00000000e-04, 6.95107991e-26, 5.98629471e-26, 3.91178589e-26,
       4.42029414e-28, 1.45647046e+02])


In [31]:
if results.success:
    # Transform the coefficients back to the context of original features 
    coeffs[0] = results.x[0]
    coeffs[focal_coeff_idx] = results.x[focal_coeff_idx]
    
    for i in range(focal_coeff_idx + 1, len(features) + 1):
        coeffs[i] = coeffs[i - 1] * (1 + min_gap_pct[i - 1]) + results.x[i]
    
    i = focal_coeff_idx - 1
    while i:
        coeffs[i] = coeffs[i + 1] / (1 + min_gap_pct[i]) - results.x[i]
        i -= 1
    
    # Finalize
    for i in range(1, len(features) + 1):
        if coeffs[i] < min_con_orig[i]:
            coeffs[i] = min_con_orig[i]
            if i < len(features) and coeffs[i + 1] < coeffs[i] * (1 + min_gap_pct[i]):
                coeffs[i + 1] = coeffs[i] * (1 + min_gap_pct[i])
        elif coeffs[i] > max_con_orig[i]:
            coeffs[i] = max_con_orig[i]
            if i > 1 and coeffs[i - 1] > coeffs[i] / (1 + min_gap_pct[i - 1]):
                coeffs[i - 1] = coeffs[i] / (1 + min_gap_pct[i - 1])
    
    # Validate
    tol = 0.001
    for i in range(1, len(features) + 1):
        if min_con_orig[i] - coeffs[i] > tol or coeffs[i] - max_con_orig[i] > tol or\
        (i < len(features) and coeffs[i] * (1 + min_gap_pct[i]) - coeffs[i + 1] > tol) or\
        (i > 1 and coeffs[i - 1] - coeffs[i] / (1 + min_gap_pct[i - 1]) > tol):
            print("Convergence was not achieved!", coeffs)
            break
    print("Final Coefficients (including intercept):", coeffs)
else:
    print("Convergence was not achieved!")

Final Coefficients (including intercept): [9.999999999999999e-05, 76.02957579127217, 85.91342064413753, 129.72926517264767, 141.404899038186, 145.64704600933158]
